<a href="https://colab.research.google.com/github/calebcaptain1/Twitter-Reviews-Naive-Bayes/blob/main/Twitter_Airline_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [135]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
import os
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds
##import tldextract
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras import callbacks
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
import keras

In [2]:
dataset_url = 'https://github.com/calebcaptain1/Twitter-Reviews-Naive-Bayes/archive/main.zip'

datapath = tf.keras.utils.get_file('Twitter-Reviews-Naive-Bayes-main.zip',
                                      cache_subdir=os.path.abspath('.'),
                                      origin = dataset_url,
                                      extract = True)

data = os.path.abspath('Twitter-Reviews-Naive-Bayes-main')
tweets = '/Tweets.csv'


 417792/Unknown - 0s 1us/step 

In [4]:
data
path = data + tweets
print(path)
tweetdata = pd.read_csv(path)

/content/Twitter-Reviews-Naive-Bayes-main/Tweets.csv


In [5]:
tweetdata.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [6]:
tweetdata.columns

Index(['tweet_id', 'airline_sentiment', 'airline_sentiment_confidence',
       'negativereason', 'negativereason_confidence', 'airline',
       'airline_sentiment_gold', 'name', 'negativereason_gold',
       'retweet_count', 'text', 'tweet_coord', 'tweet_created',
       'tweet_location', 'user_timezone'],
      dtype='object')

In [7]:
data = tweetdata[['airline_sentiment', 'airline', 'text']]

In [8]:
data.head()

,airline_sentiment,airline,text
0,neutral,Virgin America,@VirginAmerica What @dhepburn said.
1,positive,Virgin America,@VirginAmerica plus you've added commercials t...
2,neutral,Virgin America,@VirginAmerica I didn't today... Must mean I n...
3,negative,Virgin America,@VirginAmerica it's really aggressive to blast...
4,negative,Virgin America,@VirginAmerica and it's a really big bad thing...


In [9]:
data['text'] = data['text'].str.lower()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [10]:
data['text']

0                      @virginamerica what @dhepburn said.
1        @virginamerica plus you've added commercials t...
2        @virginamerica i didn't today... must mean i n...
3        @virginamerica it's really aggressive to blast...
4        @virginamerica and it's a really big bad thing...
                               ...                        
14635    @americanair thank you we got on a different f...
14636    @americanair leaving over 20 minutes late flig...
14637    @americanair please bring american airlines to...
14638    @americanair you have my money, you change my ...
14639    @americanair we have 8 ppl so we need 2 know h...
Name: text, Length: 14640, dtype: object

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()  
counts = count_vect.fit_transform(data['text'])

from sklearn.feature_extraction.text import TfidfTransformer

transformer = TfidfTransformer().fit(counts)

counts = transformer.transform(counts)  

In [12]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

le.fit(data['airline_sentiment'])

LabelEncoder()

In [13]:
le.classes_

array(['negative', 'neutral', 'positive'], dtype=object)

In [14]:
le.transform(data['airline_sentiment'])

array([1, 2, 1, ..., 1, 0, 1])

In [15]:
data['airline_sentiment'] = le.transform(data['airline_sentiment'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [16]:
data['airline_sentiment']

0        1
1        2
2        1
3        0
4        0
        ..
14635    2
14636    0
14637    1
14638    0
14639    1
Name: airline_sentiment, Length: 14640, dtype: int64

In [79]:
from sklearn.model_selection import train_test_split, GridSearchCV

In [96]:
x_train, x_test, y_train, y_test = train_test_split(counts, data['airline_sentiment'], test_size = .2, random_state=42)

In [97]:
from sklearn.naive_bayes import MultinomialNB

In [98]:
x_train.shape

(11712, 15051)

In [99]:
y_train.shape

(11712,)

In [100]:
model = MultinomialNB()

In [101]:
model.fit(x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [102]:
pred = model.predict(x_test)

In [103]:
model.score(x_test, y_test)

0.6892076502732241

In [104]:
from sklearn.metrics import confusion_matrix, classification_report

In [105]:
confusion_matrix(y_test, pred)

array([[1881,    7,    1],
       [ 493,   83,    4],
       [ 390,   15,   54]])

In [106]:
print(classification_report(y_test, pred, digits=4))

              precision    recall  f1-score   support

           0     0.6805    0.9958    0.8085      1889
           1     0.7905    0.1431    0.2423       580
           2     0.9153    0.1176    0.2085       459

    accuracy                         0.6892      2928
   macro avg     0.7954    0.4188    0.4198      2928
weighted avg     0.7391    0.6892    0.6023      2928



In [107]:
np.mean(y_test == pred)

0.6892076502732241

Sooooo, looks like it has a mean accuracy score of about 67 ~ 68%! Not too bad, however, I wonder if I can change it by messing with the passed parameters.

In [108]:
x_train, x_test, y_train, y_test = train_test_split(counts, data['airline_sentiment'], test_size = .2, random_state=42)

In [109]:
secondmod = MultinomialNB(alpha=2)

In [110]:
secondmod.fit(x_train, y_train)

MultinomialNB(alpha=2, class_prior=None, fit_prior=True)

In [111]:
secondmod.score(x_test, y_test)

0.6659836065573771

Biggest of oofs. It actually went down.

In [112]:
secondmod = MultinomialNB(alpha=.5)

In [113]:
secondmod.fit(x_train, y_train)

MultinomialNB(alpha=0.5, class_prior=None, fit_prior=True)

In [114]:
secondmod.score(x_test, y_test)

0.7274590163934426

So: turns out, less smoothing would actually be BETTER for the model! Lets put it in a loop and display some values. One more thing to try: we've been trying all of the models with fit_prior = TRUE. Time to try it with FALSE!

In [115]:
secondmod = MultinomialNB(alpha=0, fit_prior=True)
secondmod.fit(x_train, y_train)
secondmod.score(x_test, y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:507: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


0.7462431693989071

In [116]:
secondmod = MultinomialNB(alpha=0, fit_prior=False)
secondmod.fit(x_train, y_train)
secondmod.score(x_test, y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:507: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


0.710724043715847

So, with our experiment, we do want the model to process and learn prior class probabilities. AKA: fit_prior = True is much better.

In [117]:
alpha_vals = [1, .5, .25, .125, .0625, .03125, .015625, .0078125, 0]
table = {}

for alphai in alpha_vals:
  model = MultinomialNB(alpha=alphai)
  model.fit(x_train, y_train)
  mean_score = model.score(x_test, y_test)
  table[alphai] = mean_score

/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:507: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


I figured out that the sweet spot of really good values is in between .03125 and .125 So its time to figure out where the best value is in between these values.

In [118]:
table

{0: 0.7462431693989071,
 0.0078125: 0.7609289617486339,
 0.015625: 0.766051912568306,
 0.03125: 0.76775956284153,
 0.0625: 0.7670765027322405,
 0.125: 0.7674180327868853,
 0.25: 0.7544398907103825,
 0.5: 0.7274590163934426,
 1: 0.6892076502732241}

*Close* to a 77% accuracy, however, I want to figure out if the values have a better peak.

In [138]:
model = MultinomialNB()

new_vals = {
    'alpha' :  [.03125, .0625, .125]
}

optimized_model = GridSearchCV(model, new_vals)




In [139]:
optimized_model.fit(x_train, y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=MultinomialNB(alpha=1.0, class_prior=None,
                                     fit_prior=True),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': [0.03125, 0.0625, 0.125]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [140]:
print(classification_report(y_test, optimized_model.predict(x_test), digits=4))

              precision    recall  f1-score   support

           0     0.7932    0.9381    0.8596      1889
           1     0.6188    0.3862    0.4756       580
           2     0.7530    0.5447    0.6321       459

    accuracy                         0.7671      2928
   macro avg     0.7217    0.6230    0.6558      2928
weighted avg     0.7523    0.7671    0.7478      2928



In [142]:
optimized_model.cv_results_

{'mean_fit_time': array([0.00695558, 0.00672865, 0.00707798]),
 'mean_score_time': array([0.00102663, 0.00107956, 0.0011097 ]),
 'mean_test_score': array([0.74837689, 0.74914535, 0.74564517]),
 'param_alpha': masked_array(data=[0.03125, 0.0625, 0.125],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'alpha': 0.03125}, {'alpha': 0.0625}, {'alpha': 0.125}],
 'rank_test_score': array([2, 1, 3], dtype=int32),
 'split0_test_score': array([0.75842936, 0.75842936, 0.75245412]),
 'split1_test_score': array([0.74818609, 0.7494665 , 0.74263764]),
 'split2_test_score': array([0.75234842, 0.74893254, 0.74722459]),
 'split3_test_score': array([0.75320239, 0.75704526, 0.75277541]),
 'split4_test_score': array([0.72971819, 0.73185312, 0.73313407]),
 'std_fit_time': array([0.00138795, 0.00050695, 0.00058071]),
 'std_score_time': array([8.16760949e-05, 8.09035429e-05, 8.27559921e-05]),
 'std_test_score': array([0.00988225, 0.00946331, 0.0072838

In [141]:
print(optimized_model.best_params_)

optimized_model.best_score_

{'alpha': 0.0625}


0.7491453547514937

Looks Like .0625 wins with an averaged Best Score of almost 75%. Thats pretty good!

In [ ]:
data['airline'].unique()

array(['Virgin America', 'United', 'Southwest', 'Delta', 'US Airways',
       'American'], dtype=object)

In [ ]:
Airlines = ['Virgin America', 'United', 'Southwest', 'Delta', 'US Airways',
       'American']

data['airline'].value_counts(normalize=True)

data.groupby('airline').mean()[['airline_sentiment']] * 100

,airline_sentiment
airline,
American,41.138094
Delta,81.503150
Southwest,74.545455
US Airways,31.548232
United,43.982208
Virgin America,94.246032


Looks like Virgin America has the best sentiment towards it, based on the data. 